In [1]:
import torch
import numpy as np
import toydiff as tdf

In [26]:
def generate_input(shape, n_tensors=1):
    def create(shape):
        arr = np.random.rand(*shape)
        tensor = tdf.Tensor(arr, track_gradient=True)
        torch_tensor = torch.Tensor(arr)
        torch_tensor.requires_grad = True
        return tensor, torch_tensor

    tensors = []
    for _ in (0, n_tensors):
        tensors.append(create(shape))
    return tensors

In [28]:
RTOL = 1e-6

In [31]:
tensor

Tensor([[0.67767465, 0.18214098, 0.4775876 , 0.98072237, 0.5492775 ],
       [0.3858788 , 0.4519953 , 0.18095264, 0.98665076, 0.89528   ],
       [0.5988617 , 0.09006644, 0.4383224 , 0.2665586 , 0.08997621],
       [0.8634823 , 0.08804896, 0.44418818, 0.29879487, 0.6032249 ],
       [0.73410827, 0.5598107 , 0.18564452, 0.08995432, 0.78443724]],
      dtype=float32, track_gradient=True)

In [41]:
out.numpy().ndim

0

In [37]:
np.expand_dims(out.numpy(), 0)

array([0.4761456], dtype=float32)

In [30]:
# test 2d
tensor, tensor_torch = generate_input((5, 5))[0]
out = tdf.mean(tensor)
out_torch = torch.mean(tensor_torch)

# call backward
out.backward()
out_torch.backward(torch.ones_like(out_torch))

# test forward
np.testing.assert_allclose(
    out.numpy(), out_torch.detach().numpy(), rtol=RTOL
)

# test backward
np.testing.assert_allclose(
    tensor.gradient.numpy(), tensor_torch.grad.numpy(), rtol=RTOL
)

AxisError: axis 1 is out of bounds for array of dimension 1

In [21]:
d = tdf.rand((2, 2, 2, 2), track_gradient=True)
b = tdf.mean(d, axis=2)
b.backward()

In [25]:
d.gradient

Tensor([[[[0.5, 0.5],
         [0.5, 0.5]],

        [[0.5, 0.5],
         [0.5, 0.5]]],


       [[[0.5, 0.5],
         [0.5, 0.5]],

        [[0.5, 0.5],
         [0.5, 0.5]]]], dtype=float32, track_gradient=False)

In [19]:
td = torch.rand(2, 2, 2, 2)
td.requires_grad = True

tb = torch.mean(td, dim=2)
tb.backward(torch.ones_like(tb))

In [24]:
td.grad

tensor([[[[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]]],


        [[[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]]]])